# Reto 03
Une el ejemplo 01, 02, 03 y 04 en un solo cuaderno: 
- Inicializa una población.
- Realiza la cruza de sus individuos.
- Realiza la mutación de la población.
- Evalúe la población.
- Filtre a los individuos mas capaces.

Si lograste hacerlo, deberás ser capaz de generar una población, cruzarla, mutarla, evaluarla y quedarte con los individuos mas capaces. Deberás repetir todas las operaciones en ciclo (excepto la de generar una población) para obtener la mejor solución.

In [1]:
import numpy as np
import random as rnd

### Funciones del ejemplo 01.

In [2]:
def inicializarIndividuo(individuo):
    total = 1.0
    cantidadGenes = len(individuo)
    #Por cada uno de los genes...
    for counter in range(0,cantidadGenes):
        #Si llegaste al último gen, pon lo que sobre a este gen.
        if(counter == cantidadGenes -1):
            individuo[counter] = total
        #Sino, 
        else:  
            #obten un valor aleatorio entre 0 y lo que quede del 100%
            randomValue = rnd.uniform(0,total)
            #Dale ese valor aleatorio a ese gen.
            individuo[counter] = randomValue
            #Y resta lo que obtuviste del 100% original.
            total -=randomValue
    return individuo

def generarPoblacion(cantidadDeIndividuos, cantidadDeGenes):
    #reservamos la memoria que usarán los individuos: Todos los individuos nacen con genes en 0.
    poblacion = np.zeros((cantidadDeIndividuos, cantidadDeGenes))
    #Y a cada individuo se le inicializan los genes con la función "inicializar individuo"
    for counter in range(0,cantidadDeIndividuos):
        poblacion[counter] = inicializarIndividuo(poblacion[counter])
    return poblacion

### Funciones del ejemplo 02.

In [3]:
def ajustarIndividuo(individuo):
    total = np.sum(individuo)
    individuo = individuo / total
    return individuo    

def cruzarIndividuos(individuo1, individuo2):
    crossPoint = rnd.randint(1,len(individuo1)-2)
    hijo1 = np.append(individuo1[0:crossPoint], individuo2[crossPoint:len(individuo1)])
    hijo2 = np.append(individuo2[0:crossPoint], individuo1[crossPoint:len(individuo1)])
    hijo1 = ajustarIndividuo(hijo1)
    hijo2 = ajustarIndividuo(hijo2)
    return np.array([hijo1,hijo2])

def cruzarPoblacion(poblacion, probabilidadDeCruza):    
    nuevaPoblacion = None
    for counter in range(len(poblacion)):
        probabilidad = rnd.random()
        #Si la probabilidad es menor que la probabilidad de cruza, entonces se procede a la cruza.
        if(probabilidad < probabilidadDeCruza):
            #Elige pareja:
            pareja = rnd.randint(0,len(poblacion)-2)
            #Si al elegir una pareja, elegiste sin querer al mismo individuo, intenta de nuevo.
            while pareja == counter:
                pareja = rnd.randint(0,len(poblacion)-2)
            
            #Haz la cruza. 
            nuevosIndividuos = cruzarIndividuos(poblacion[counter],poblacion[pareja])
            
            #Añade los nuevos individuos a una lista de nuevos individuos.
            if nuevaPoblacion is None:
                nuevaPoblacion = nuevosIndividuos
            else:
                nuevaPoblacion = np.concatenate((nuevaPoblacion,nuevosIndividuos))
                
    #Al final del ciclo, si hay nuevos individuos, añadelos a la población.
    if not nuevaPoblacion is None:
        poblacion = np.concatenate((poblacion, nuevaPoblacion))
    return poblacion

### Funciones del ejemplo 03.

In [4]:
def mutacion(individuo):
    mutationPoint = rnd.randint(1,len(individuo)-2)
    mutationValue = rnd.uniform(0.1, 0.3)
    individuoMutante = np.array(individuo, copy=True)
    individuoMutante[mutationPoint] +=mutationValue
    individuoMutante = ajustarIndividuo(individuoMutante)
    return np.array([individuoMutante])

def mutarPoblacion(poblacion, probabilidadDeMutacion):
    nuevaPoblacion = None
    for counter in range(len(poblacion)):
        probabilidad = rnd.random()
        if(probabilidad < probabilidadDeMutacion):
            nuevoMutante = mutacion(poblacion[counter])
            if nuevaPoblacion is None:
                nuevaPoblacion = nuevoMutante
            else:
                nuevaPoblacion = np.concatenate((nuevaPoblacion,nuevoMutante))
    if not nuevaPoblacion is None:        
        poblacion = np.concatenate((poblacion, nuevaPoblacion))
    return poblacion

### Funciones del ejemplo 04.

In [5]:
def evaluacion(poblacion, ganancias, probabilidadDeExito):
    puntaje = np.zeros(poblacion.shape[0])

    puntajeDeGanancia = np.dot(poblacion, ganancias)
    puntajeDeExito = np.dot(poblacion, probabilidadDeExito)
    puntaje = puntajeDeGanancia+puntajeDeExito
    
    ordenDeMayorAMenor = np.argsort(-puntaje)
    poblacionOrdenada = poblacion[ordenDeMayorAMenor]
    mejorPuntaje = (np.sort(-puntaje)[0])*-1
    return poblacionOrdenada, mejorPuntaje

def eliminarIndividuosPorPorcentaje(poblacion, porcentajeAMantener):
    cantidad = poblacion.shape[0]
    cantidadAMantener = int(porcentajeAMantener * cantidad)
    return poblacion[:cantidadAMantener]
    
def eliminarIndividuosPorCantidad(poblacion, cantidadAMantener):    
    return poblacion[:cantidadAMantener]

### Unión de todos los ejemplos: Algoritmo genético.

In [6]:
poblacion = generarPoblacion(10,4)
print("Poblacion original:")
print(poblacion)

ganancias = [0.1,0.1,0.9,0.1]
probabilidadDeExito = [0.1,0.1,0.9,0.1]

totalGeneraciones = 100

for generacion in range (0, totalGeneraciones):
    poblacion = cruzarPoblacion(poblacion, 0.3)
    poblacion = mutarPoblacion(poblacion,0.3)
    poblacion,mejorPuntaje= evaluacion(poblacion,ganancias, probabilidadDeExito)
    poblacion = eliminarIndividuosPorCantidad(poblacion, 5)
    print("Mejor puntaje de la generación "+str(generacion)+":"+str(mejorPuntaje))
    
print("Solución: "+str(poblacion[0]))

Poblacion original:
[[7.54038889e-01 6.98760445e-02 5.99636005e-02 1.16121466e-01]
 [3.76754194e-01 2.72171873e-02 9.00619904e-02 5.05966628e-01]
 [3.78608988e-01 4.22840001e-01 1.80898466e-01 1.76525442e-02]
 [9.82485546e-02 3.88008751e-02 8.17111191e-01 4.58393789e-02]
 [9.58648960e-01 1.21333711e-02 2.15175480e-02 7.70012076e-03]
 [9.67973904e-01 1.93250935e-02 8.29581904e-03 4.40518343e-03]
 [1.56226853e-01 1.27642383e-01 9.85108923e-02 6.17619872e-01]
 [1.91778121e-02 5.99625104e-01 1.48853581e-01 2.32343503e-01]
 [4.72490345e-01 1.55007155e-01 1.74935885e-01 1.97566616e-01]
 [8.81429278e-01 5.34149465e-02 8.11343022e-04 6.43444320e-02]]
Mejor puntaje de la generación 0:1.5073779063223447
Mejor puntaje de la generación 1:1.5589887791301384
Mejor puntaje de la generación 2:1.5589887791301384
Mejor puntaje de la generación 3:1.5589887791301384
Mejor puntaje de la generación 4:1.583621940714826
Mejor puntaje de la generación 5:1.583621940714826
Mejor puntaje de la generación 6:1.5836